In [ ]:
class Test_Agent2:
    def __init__(self, input_dim, output_dim, model):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.model = model
        self.epsilon = 0.0

    def get_action(self, state):

        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        with torch.no_grad():
            q_values = self.model(state)

        action = torch.argmax(q_values, dim=1).item()
        return action

In [ ]:
def test_model2(model_path, agent, env, num_episodes=10000):
    episodes, scores, score_avg, test_action, win_list, win_rate_list = [], [], [], [], [], []
    agent.model.load_state_dict(torch.load(model_path))
    agent.model.eval()

    for epi in range(num_episodes):
        done = False
        score = 0
        test_action_epi = []

        state = env.reset()
        while not done:
            action = agent.get_action(state)
            if action not in test_action_epi:
              test_action_epi.append(action)
              next_state, reward, exploded, done = env.step(action)
              score += reward
              state = next_state
            else:
              win_list.append(0)
              break

        if done:
            # win
            if not env.exploded:
                win_list.append(1)
            else:
                win_list.append(0)

            test_action.append(len(test_action_epi))
            episodes.append(epi)
            scores.append(score)
            score_avg.append(np.mean(scores[-100:]))

        if (epi % 100) == 0:

          win_rate = np.mean(win_list[-100:]) if len(win_list) >= 100 else np.mean(win_list)
          action_avg = np.mean(test_action[-100:]) if len(test_action) >= 100 else np.mean(test_action)
          win_rate_list.append(win_rate)
          print(f"Episode: {epi} | Score: {np.median(scores[-100:]):.5f} | Win Rate: {win_rate:.2f} | action_list: {action_avg:.2f}")

    print(f"Win rate: {np.mean(win_list)} | Standard Deviation: {np.std(win_rate_list)}")

env = MinesweeperEnv()
model = NeuralNet(env.height * env.width)
agent = Test_Agent2(env.height * env.width, env.height * env.width, model)

model_path = '/content/drive/MyDrive/minesweeper_model2.pth'
test_model2(model_path, agent, env, num_episodes=10000)